## A note book to review and summarize QC metrics for HRApop v1.0

## Install and import libraries

In [1]:
%pip install requests pandas seaborn 

import requests
import pandas
import seaborn

Note: you may need to restart the kernel to use updated packages.


## Load data